# Data Load

In [ ]:
# Importing Libraries
import pandas as pd

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Changing the work directory
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [ ]:
# Load the Market Data and Fill CSVs
md_df = pd.read_csv('MD-Combined.csv')

fill_df = pd.read_csv('Fills-Combined.csv')

# Combining the two DataFrames

In [ ]:
# Convert 'ExchangePublishTimestamp' and 'ts' columns to datetime objects
fill_df['ExchangePublishTimestamp']=pd.to_datetime(fill_df['ExchangePublishTimestamp'])
md_df['ts']=pd.to_datetime(md_df['ts'])

# Ensure 'Symbol' column is of the same type in both dataframes
fill_df['Symbol'] = fill_df['Symbol'].astype(str) #Changing them both to strings
md_df['Symbol'] = md_df['Symbol'].astype(str)

# Merging
md_fill_df = pd.merge_asof(fill_df.sort_values('ExchangePublishTimestamp'),
                           md_df.sort_values('ts'), left_on='ExchangePublishTimestamp', right_on='ts',
                           by='Symbol', direction='nearest')

# Slippage Analysis

In [ ]:
# Calculate mid-price
md_fill_df['Mid_Price'] = (md_fill_df['bid_price'] + md_fill_df['ask_price']) / 2

# Calculate Slippage
md_fill_df['Slippage'] = abs(md_fill_df['Mid_Price'] - md_fill_df['Price_x'])/md_fill_df['Mid_Price'] * 100 #In %

# Slippage by Firm and Trader
slippage_by_firm = md_fill_df.groupby('Firm')['Slippage'].mean()
slippage_by_firm=slippage_by_firm.sort_values(ascending=True)
slippage_by_trader = md_fill_df.groupby('Trader_x')['Slippage'].mean()
slippage_by_trader=slippage_by_trader.sort_values(ascending=True)

print("Average Slippage By Firm (%): ")
print(slippage_by_firm)
print("\nAverage Slippage By Trader (%): ")
print(slippage_by_trader)

Average Slippage By Firm (%): 
Firm
EIN    0.001538
NJG    0.001596
ABW    0.001735
YDG    0.001748
UJV    0.001752
KNR    0.001811
IOU    0.001822
SAU    0.001995
Name: Slippage, dtype: float64

Average Slippage By Trader (%): 
Trader_x
TIE    0.001038
PUL    0.001044
NBM    0.001052
MHP    0.001055
UFQ    0.001060
YHL    0.001063
KQD    0.001064
TCF    0.001077
GXU    0.001078
OTY    0.001081
YWX    0.001081
JZF    0.001086
RVN    0.001089
MVL    0.001091
NVF    0.001096
PAM    0.001109
JGS    0.001112
ZGS    0.002203
EBJ    0.002210
VHA    0.002213
JFX    0.002213
GWS    0.002213
LYW    0.002214
BJA    0.002214
ASY    0.002216
YLO    0.002219
WNT    0.002220
SCK    0.002221
VRP    0.002222
HFW    0.002222
LFH    0.002224
KOX    0.002225
Name: Slippage, dtype: float64


We see from the above calculations that they all have very low slippage which indicates better execution relative to the market. 'EIN' has the best numbers among the firms while 'TIE' is the best among the traders.

# Bid-Ask Spread Impact

In [ ]:
# Calculate Spread
md_fill_df['Spread'] = md_fill_df['ask_price'] - md_fill_df['bid_price']

# Spread by symbol
spread_by_symbol = md_fill_df.groupby('Symbol')['Spread'].mean()

# Slippage vs Spread by Order Type
slippage_vs_spread = md_fill_df.groupby(['IsBuy_x','IsAggressor'])[['Slippage','Spread']].mean()

print("Average Spread By Symbol: ")
print(spread_by_symbol)
print("\nSlippage vs Spread By Order Type: ")
print(slippage_vs_spread)

Average Spread By Symbol: 
Symbol
RAINBOWZ0    26.689566
UNICORNZ0    52.103327
Name: Spread, dtype: float64

Slippage vs Spread By Order Type: 
                     Slippage     Spread
IsBuy_x IsAggressor                     
False   False        0.001773  33.997416
        True         0.001635  39.788027
True    False        0.001864  38.536678
        True         0.001603  37.133311


We see that 'UNICORNZ0' has the wider spread. Almost double of 'RAINBOWZ0'. This could explain why more trades happen for the latter symbol than the former. As there are more participants for 'RAINBOWZ0', the gap between the bid and ask is tighter.
We see that aggressive orders have a wider spread for sell orders while it's the opposite for buy orders even though the difference in the aggressive and passive approach is not much. Interesting thing is Spread and Slippage have an inversely proportional relationship except in the case of buy-passive orders. We have already deduced previously that the market has more neutral sentiment and there's a demand and supply mismatch. Traders are approaching with caution and while overall uncertainty might be higher, the market is relatively stable during the brief window of order execution.

# Liquidity Analysis

In [ ]:
# Liquity by symbol
liquidity_by_symbol = md_fill_df.groupby('Symbol')[['bid_qty','ask_qty']].mean()

print("Average Liquidity By Symbol: ")
print(liquidity_by_symbol)

Average Liquidity By Symbol: 
           bid_qty    ask_qty
Symbol                       
RAINBOWZ0      NaN  59.554982
UNICORNZ0      NaN   3.297197


Unfortunately, we couldn't get any value for bid_qty but 'UNICORNZ0' has way lower ask_qty mean, which might explain higher slippage and spread for that symbol. Less liquidity can lead to higher spread due to fewer participants and increased risk for market makers. The lack of participation leads to less competition among market makers, allowing them to offer wider bid-ask spreads to increase their profitability and compensate for higher risk.